# Tagging and Extraction Using OpenAI functions

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
from typing import List
from pydantic import BaseModel, Field
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

In [3]:
class Tagging(BaseModel):
    """Tag the piece of text with particular info."""
    sentiment: str = Field(description="sentiment of text, should be `pos`, `neg`, or `neutral`")
    language: str = Field(description="language of text (should be ISO 639-1 code)")

In [4]:
convert_pydantic_to_openai_function(Tagging)

/var/folders/dv/gzhyqctn53s9bh23g7tbvl940000gn/T/ipykernel_3828/1725091243.py:1: LangChainDeprecationWarning: The function `convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use :meth:`~langchain_core.utils.function_calling.convert_to_openai_function()` instead.
  convert_pydantic_to_openai_function(Tagging)


{'name': 'Tagging',
 'description': 'Tag the piece of text with particular info.',
 'parameters': {'properties': {'sentiment': {'description': 'sentiment of text, should be `pos`, `neg`, or `neutral`',
    'type': 'string'},
   'language': {'description': 'language of text (should be ISO 639-1 code)',
    'type': 'string'}},
  'required': ['sentiment', 'language'],
  'type': 'object'}}

In [5]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

In [6]:
model = ChatOpenAI(temperature=0)

/var/folders/dv/gzhyqctn53s9bh23g7tbvl940000gn/T/ipykernel_3828/1167291718.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI(temperature=0)


In [7]:
tagging_functions = [convert_pydantic_to_openai_function(Tagging)]

In [8]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Think carefully, and then tag the text as instructed"),
    ("user", "{input}")
])

In [9]:
model_with_functions = model.bind(
    functions=tagging_functions,
    function_call={"name": "Tagging"}
)

In [10]:
tagging_chain = prompt | model_with_functions

In [11]:
tagging_chain.invoke({"input": "I love langchain"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"sentiment":"pos","language":"en"}', 'name': 'Tagging'}}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 108, 'total_tokens': 118, 'completion_tokens_details': {'reasoning_tokens': 0}, 'prompt_tokens_details': {'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0a976960-11a4-4bf3-bf70-ec3623553a56-0')

In [16]:
tagging_chain.invoke({"input": "non mi piace questo cibo"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"sentiment":"neg","language":"it"}', 'name': 'Tagging'}}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 111, 'total_tokens': 121, 'completion_tokens_details': {'reasoning_tokens': 0}, 'prompt_tokens_details': {'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ec52ac89-5902-4205-9afa-8671a825459f-0')

In [69]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

In [70]:
tagging_chain = prompt | model_with_functions | JsonOutputFunctionsParser()

In [71]:
tagging_chain.invoke({"input": "non mi piace questo cibo"})

{'sentiment': 'neg', 'language': 'it'}

## Extraction

Extraction is similar to tagging, but used for extracting multiple pieces of information.

In [72]:
from typing import Optional
class Person(BaseModel):
    """Information about a person."""
    name: str = Field(description="person's name")
    age: Optional[int] = Field(description="person's age")

In [73]:
class Information(BaseModel):
    """Information to extract."""
    people: List[Person] = Field(description="List of info about people")

In [74]:
convert_pydantic_to_openai_function(Information)

{'name': 'Information',
 'description': 'Information to extract.',
 'parameters': {'properties': {'people': {'description': 'List of info about people',
    'items': {'description': 'Information about a person.',
     'properties': {'name': {'description': "person's name", 'type': 'string'},
      'age': {'anyOf': [{'type': 'integer'}, {'type': 'null'}],
       'description': "person's age"}},
     'required': ['name', 'age'],
     'type': 'object'},
    'type': 'array'}},
  'required': ['people'],
  'type': 'object'}}

In [75]:
extraction_functions = [convert_pydantic_to_openai_function(Information)]
extraction_model = model.bind(functions=extraction_functions, function_call={"name": "Information"})

In [77]:
extraction_model.invoke("Joe is 30, his mom is Martha")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"people":[{"name":"Joe","age":30},{"name":"Martha","age":null}]}', 'name': 'Information'}}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 95, 'total_tokens': 116, 'completion_tokens_details': {'reasoning_tokens': 0}, 'prompt_tokens_details': {'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a5818912-f15a-4360-a8c4-5e1b2a6e48b5-0')

In [78]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract the relevant information, if not explicitly provided do not guess. Extract partial info"),
    ("human", "{input}")
])

In [79]:
extraction_chain = prompt | extraction_model

In [80]:
extraction_chain.invoke({"input": "Joe is 30, his mom is Martha"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"people":[{"name":"Joe","age":30},{"name":"Martha","age":null}]}', 'name': 'Information'}}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 112, 'total_tokens': 133, 'completion_tokens_details': {'reasoning_tokens': 0}, 'prompt_tokens_details': {'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-05835fb9-5063-45d0-bfaa-a1a4eb20030b-0')

In [81]:
extraction_chain = prompt | extraction_model | JsonOutputFunctionsParser()

In [82]:
extraction_chain.invoke({"input": "Joe is 30, his mom is Martha"})

{'people': [{'name': 'Joe', 'age': 30}, {'name': 'Martha', 'age': None}]}

In [83]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

In [84]:
extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="people")

In [85]:
extraction_chain.invoke({"input": "Joe is 30, his mom is Martha"})

[{'name': 'Joe', 'age': 30}, {'name': 'Martha', 'age': None}]

## Doing it for real

We can apply tagging to a larger body of text.

For example, let's load this blog post and extract tag information from a sub-set of the text.

In [88]:
from langchain.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
documents = loader.load()

In [89]:
doc = documents[0]

In [90]:
page_content = doc.page_content[:10000]

In [91]:
print(page_content[:1000])







LLM Powered Autonomous Agents | Lil'Log







































Lil'Log






















Posts




Archive




Search




Tags




FAQ




emojisearch.app









      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


 


Table of Contents



Agent System Overview

Component One: Planning

Task Decomposition

Self-Reflection


Component Two: Memory

Types of Memory

Maximum Inner Product Search (MIPS)


Component Three: Tool Use

Case Studies

Scientific Discovery Agent

Generative Agents Simulation

Proof-of-Concept Examples


Challenges

Citation

References





Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general

In [92]:
class Overview(BaseModel):
    """Overview of a section of text."""
    summary: str = Field(description="Provide a concise summary of the content.")
    language: str = Field(description="Provide the language that the content is written in.")
    keywords: str = Field(description="Provide keywords related to the content.")

In [93]:
overview_tagging_function = [
    convert_pydantic_to_openai_function(Overview)
]
tagging_model = model.bind(
    functions=overview_tagging_function,
    function_call={"name":"Overview"}
)
tagging_chain = prompt | tagging_model | JsonOutputFunctionsParser()

In [94]:
tagging_chain.invoke({"input": page_content})

{'summary': 'The article discusses building autonomous agents powered by LLM (large language model) with components like planning, memory, and tool use. It also covers techniques like task decomposition, self-reflection, and challenges faced in implementing LLM-powered agents.',
 'language': 'English',
 'keywords': 'LLM, autonomous agents, planning, memory, tool use, task decomposition, self-reflection, challenges'}

In [95]:
class Paper(BaseModel):
    """Information about papers mentioned."""
    title: str
    author: Optional[str]


class Info(BaseModel):
    """Information to extract"""
    papers: List[Paper]

In [96]:
paper_extraction_function = [
    convert_pydantic_to_openai_function(Info)
]
extraction_model = model.bind(
    functions=paper_extraction_function, 
    function_call={"name":"Info"}
)
extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="papers")

In [97]:
extraction_chain.invoke({"input": page_content})

[{'title': 'LLM Powered Autonomous Agents', 'author': 'Lilian Weng'}]

In [98]:
template = """A article will be passed to you. Extract from it all papers that are mentioned by this article. 

Do not extract the name of the article itself. If no papers are mentioned that's fine - you don't need to extract any! Just return an empty list.

Do not make up or guess ANY extra information. Only extract what exactly is in the text."""

prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", "{input}")
])

In [99]:
extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="papers")

In [100]:
extraction_chain.invoke({"input": page_content})

[{'title': 'Chain of thought (CoT; Wei et al. 2022)', 'author': None},
 {'title': 'Tree of Thoughts (Yao et al. 2023)', 'author': None},
 {'title': 'LLM+P (Liu et al. 2023)', 'author': None},
 {'title': 'ReAct (Yao et al. 2023)', 'author': None},
 {'title': 'Reflexion (Shinn & Labash 2023)', 'author': None},
 {'title': 'Chain of Hindsight (CoH; Liu et al. 2023)', 'author': None},
 {'title': 'Algorithm Distillation (AD; Laskin et al. 2023)', 'author': None}]

In [101]:
extraction_chain.invoke({"input": "hi"})

[]

In [102]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_overlap=0)

In [103]:
splits = text_splitter.split_text(doc.page_content)

In [104]:
len(splits)

15

In [105]:
def flatten(matrix):
    flat_list = []
    for row in matrix:
        flat_list += row
    return flat_list

In [106]:
flatten([[1, 2], [3, 4]])

[1, 2, 3, 4]

In [107]:
print(splits[0])

LLM Powered Autonomous Agents | Lil'Log







































Lil'Log






















Posts




Archive




Search




Tags




FAQ




emojisearch.app









      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


 


Table of Contents



Agent System Overview

Component One: Planning

Task Decomposition

Self-Reflection


Component Two: Memory

Types of Memory

Maximum Inner Product Search (MIPS)


Component Three: Tool Use

Case Studies

Scientific Discovery Agent

Generative Agents Simulation

Proof-of-Concept Examples


Challenges

Citation

References





Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general probl

In [108]:
from langchain.schema.runnable import RunnableLambda

In [109]:
prep = RunnableLambda(
    lambda x: [{"input": doc} for doc in text_splitter.split_text(x)]
)

In [110]:
prep.invoke("hi")

[{'input': 'hi'}]

In [111]:
chain = prep | extraction_chain.map() | flatten

In [112]:
chain.invoke(doc.page_content)

[{'title': 'AutoGPT', 'author': None},
 {'title': 'GPT-Engineer', 'author': None},
 {'title': 'BabyAGI', 'author': None},
 {'title': 'Chain of thought', 'author': 'Wei et al. 2022'},
 {'title': 'Tree of Thoughts', 'author': 'Yao et al. 2023'},
 {'title': 'LLM+P', 'author': 'Liu et al. 2023'},
 {'title': 'ReAct', 'author': 'Yao et al. 2023'},
 {'title': 'Reflexion', 'author': 'Shinn & Labash 2023'},
 {'title': 'Chain of Hindsight', 'author': 'Liu et al. 2023'},
 {'title': 'Algorithm Distillation', 'author': 'Laskin et al. 2023'},
 {'title': 'Laskin et al. 2023', 'author': None},
 {'title': 'Miller 1956', 'author': None},
 {'title': 'Duan et al. 2017', 'author': None},
 {'title': 'LSH: Locality-Sensitive Hashing', 'author': None},
 {'title': 'ANNOY: Approximate Nearest Neighbors Oh Yeah', 'author': None},
 {'title': 'HNSW: Hierarchical Navigable Small World', 'author': None},
 {'title': 'FAISS: Facebook AI Similarity Search', 'author': None},
 {'title': 'ScaNN: Scalable Nearest Neighbors

# TRY BY YOURSELF

In [132]:
class Tagging2(BaseModel):
    """Tag the piece of text with particular info described in parameters"""
    team_name: str = Field(description="name of the team")
    qbit_type: str = Field(description="the type of qbits, should be `superconducting`, `quantum dots`, `Trapped ion`, `photonic`, `defect based`, `topological` or `nuclear magnetic resonance`")
    performance_method: str = Field(description="performance_method is the method use to assess qbits, should be in `Quantum volume`,  `CLOPS`, `Entanglement in GHz`, `QAOA`, `QSVM`")
    performance_value: str = Field(description="performance of qubits, associated with performance_method")

In [133]:
tagging_function2 = convert_pydantic_to_openai_function(Tagging2)
# tagging_function2

In [134]:
model2 = ChatOpenAI(temperature=0)

model_with_functions2 = model2.bind(functions=[tagging_function2],
                                   function_call = {"name":"Tagging2"})

prompt2 = ChatPromptTemplate.from_messages([("system","Think carefully, and tag the text as instructed"),
                                           ("user","{input}")])

tagging_chain2 = prompt2 | model_with_functions2 | JsonOutputFunctionsParser()

tagging_chain2.invoke({"input":f"exctract all the inf from here : {input[3500:]}"})

{'team_name': 'Universities',
 'qbit_type': 'neutral atom',
 'performance_method': 'scalability',
 'performance_value': 'advantageous'}

depending which part of the intpu, it gets information BUT ONLY for 1 entity == tagging

In [136]:
# we make a new class to extract a list of tagged info
class Information(BaseModel):
    """information to extract"""
    labs: List[Tagging2] = Field(description="list of work related to qbits")

convert_pydantic_to_openai_function(Information)

{'name': 'Information',
 'description': 'information to extract',
 'parameters': {'properties': {'labs': {'description': 'list of work related to qbits',
    'items': {'description': 'Tag the piece of text with particular info described in parameters',
     'properties': {'team_name': {'description': 'name of the team',
       'type': 'string'},
      'qbit_type': {'description': 'the type of qbits, should be `superconducting`, `quantum dots`, `Trapped ion`, `photonic`, `defect based`, `topological` or `nuclear magnetic resonance`',
       'type': 'string'},
      'performance_method': {'description': 'performance_method is the method use to assess qbits, should be in `Quantum volume`,  `CLOPS`, `Entanglement in GHz`, `QAOA`, `QSVM`',
       'type': 'string'},
      'performance_value': {'description': 'performance of qubits, associated with performance_method',
       'type': 'string'}},
     'required': ['team_name',
      'qbit_type',
      'performance_method',
      'performance_v

In [137]:
# we make another extraction function to see if model can choose the good one 
# taggin3 not converte to openai function because extraction function wil use it
class Tagging3(BaseModel):
    """Tag the piece of text with particular info described in parameters"""
    univesrity: str = Field(description="name of university")
    doctoral_program: float = Field(description="name of doctoral program if related to life science, else write NaN")
    tuitions: float = Field(description="cost of tuition for 1 year study")
class Information3(BaseModel):
    """information to extract"""
    institutions: List[Tagging3] = Field(description="list of university and their life science doctoral program")


In [138]:
extration_functions = [convert_pydantic_to_openai_function(Information),convert_pydantic_to_openai_function(Information3)]

model3 = ChatOpenAI(temperature=0)
extraction_model3 = model3.bind(functions=extration_functions, function_call={"name":"Information"})


In [139]:
extraction_model3.invoke(input[3500:])

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"labs":[{"team_name":"Harvard University","qbit_type":"neutral atom","performance_method":"N/A","performance_value":"Rydberg atoms"},{"team_name":"MIT","qbit_type":"neutral atom","performance_method":"N/A","performance_value":"neutral atom arrays"},{"team_name":"Pasqal","qbit_type":"neutral atom","performance_method":"N/A","performance_value":"neutral atom processors"},{"team_name":"University of Chicago","qbit_type":"spin qubits","performance_method":"N/A","performance_value":"NV center spin qubits in diamond"},{"team_name":"Harvard University","qbit_type":"spin qubits","performance_method":"N/A","performance_value":"quantum memories and spin qubits"},{"team_name":"University of Stuttgart","qbit_type":"spin qubits","performance_method":"N/A","performance_value":"NV centers and spin qubits in diamond"},{"team_name":"Element Six","qbit_type":"spin qubits","performance_method":"N/A","performance_value":"synthetic d

In [126]:
# adds a prompt template with "system" information
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

prompt3 = ChatPromptTemplate.from_messages([
    ("system","extract relevant information, get info only from prompt, extract partial info if necessary"),
    ("user","{input}")
])

In [140]:
# instanciate a chain to use output parser
extraction_chain = prompt3 | extraction_model3 | JsonOutputFunctionsParser()

# extraction_chain.invoke(input) ## wrong way to invoke because we have a ChatPromptTemplate

extraction_chain.invoke({"input": input[-1000:]})

{'labs': [{'team_name': 'MIT',
   'qbit_type': 'neutral atom',
   'performance_method': 'Quantum volume',
   'performance_value': 'N/A'},
  {'team_name': 'Pasqal',
   'qbit_type': 'neutral atom',
   'performance_method': 'Quantum volume',
   'performance_value': 'N/A'}]}

permet de sortir juste un dictionnaire avec les infos clés

In [143]:
tagging_chain2 = prompt3 | extraction_model3 | JsonKeyOutputFunctionsParser(key_name="labs")
extraction_chain.invoke({"input": input[-1000:]})

{'labs': [{'team_name': 'MIT',
   'qbit_type': 'neutral atom',
   'performance_method': 'Quantum volume',
   'performance_value': 'N/A'},
  {'team_name': 'Pasqal',
   'qbit_type': 'neutral atom',
   'performance_method': 'Quantum volume',
   'performance_value': 'N/A'}]}

refine the system prompt

In [152]:
template = """A text will be passed to you. Extract from it all qbits types that are mentioned. 

Extract the performance_value ONLY if they are related to the performance_method. If no qbits are mentioned that's fine - you don't need to extract any! Just return an empty list.

Do not make up or guess ANY extra information. Only extract what exactly is in the text."""

prompt3 = ChatPromptTemplate.from_messages([
    ("system",template),
    ("user","{input}")
])



model4 = ChatOpenAI(temperature=0)
model_with_functions4 = model4.bind(functions=extration_functions, function_call={"name":"Information"})

extraction_chain4 = prompt3 | model_with_functions4 | JsonKeyOutputFunctionsParser(key_name="labs")

extraction_chain4.invoke({"input":input})

[{'team_name': 'Yale University',
  'qbit_type': 'superconducting',
  'performance_method': 'Quantum volume',
  'performance_value': '64'},
 {'team_name': 'University of California, Santa Barbara (UCSB)',
  'qbit_type': 'superconducting',
  'performance_method': 'CLOPS',
  'performance_value': '0.21'},
 {'team_name': 'MIT',
  'qbit_type': 'superconducting',
  'performance_method': 'Entanglement in GHz',
  'performance_value': '2.5'},
 {'team_name': 'Google Quantum AI Lab',
  'qbit_type': 'superconducting',
  'performance_method': 'QAOA',
  'performance_value': '0.85'},
 {'team_name': 'IBM',
  'qbit_type': 'superconducting',
  'performance_method': 'QSVM',
  'performance_value': '0.92'},
 {'team_name': 'University of Maryland',
  'qbit_type': 'Trapped ion',
  'performance_method': 'Quantum volume',
  'performance_value': '32'},
 {'team_name': 'University of Innsbruck, Austria',
  'qbit_type': 'Trapped ion',
  'performance_method': 'CLOPS',
  'performance_value': '0.18'},
 {'team_name': 

In [38]:
input = """Qubits (quantum bits) are the fundamental units of quantum information, and different types of qubits are being developed across various research institutions. Here’s a list of some prominent qubit types and the universities or organizations leading their development:

### 1. **Superconducting Qubits**
   - **Description**: These qubits are created using superconducting circuits that are cooled to near absolute zero. They are one of the most advanced types of qubits in current quantum computing research.
   - **Universities/Institutions**:
     - **Yale University**: Known for pioneering work in transmon qubits.
     - **University of California, Santa Barbara (UCSB)**: Focused on superconducting circuits for quantum computation.
     - **MIT**: Researches superconducting qubits and hybrid quantum systems.
     - **Google Quantum AI Lab**: Developed Sycamore, a superconducting-based quantum processor.
     - **IBM**: Leading research in superconducting qubits through IBM Q.

### 2. **Trapped Ion Qubits**
   - **Description**: These qubits use ions trapped in an electromagnetic field. The internal energy levels of these ions are manipulated using lasers to represent quantum states.
   - **Universities/Institutions**:
     - **University of Maryland**: Home to the Joint Quantum Institute (JQI), specializing in trapped ion research.
     - **University of Innsbruck, Austria**: A leader in trapped ion quantum computing.
     - **University of Oxford**: Active in trapped ion qubit research.
     - **Honeywell Quantum Solutions**: Focuses on using trapped ion technology for scalable quantum computing.

### 3. **Topological Qubits**
   - **Description**: Based on the theory of anyons and braiding, topological qubits are still largely theoretical but promise fault-tolerant quantum computation.
   - **Universities/Institutions**:
     - **Microsoft (Station Q)**: Partnered with universities like **University of Copenhagen** to work on topological qubits.
     - **University of California, Santa Barbara**: Collaborates with Microsoft on topological quantum computing.
     - **TU Delft (Delft University of Technology)**: Leading research on Majorana fermions for topological qubits.

### 4. **Photonic Qubits**
   - **Description**: These qubits encode information in the properties of photons, such as polarization. They are promising for quantum communication.
   - **Universities/Institutions**:
     - **University of Bristol**: Known for research in photonic quantum computing and quantum networks.
     - **University of Toronto**: Develops quantum photonics for quantum communication.
     - **Harvard University**: Active in quantum optics and photonic qubits.
     - **Xanadu**: A Canadian company specializing in photonic quantum computing.

### 5. **Silicon Qubits**
   - **Description**: These qubits are based on manipulating the quantum states of electrons in silicon. They offer a pathway to scale up quantum computers using traditional semiconductor manufacturing techniques.
   - **Universities/Institutions**:
     - **University of New South Wales (UNSW), Australia**: Leading research in silicon quantum computing.
     - **University of Wisconsin–Madison**: Researches quantum dots and silicon qubits.
     - **Intel**: Through collaboration with **QuTech** (TU Delft), Intel is working on silicon spin qubits.

### 6. **Neutral Atom Qubits**
   - **Description**: Neutral atoms are trapped in optical lattices and used to represent quantum states. These systems are advantageous due to their scalability.
   - **Universities/Institutions**:
     - **Harvard University**: Researching neutral atom quantum computing, especially with Rydberg atoms.
     - **MIT**: Focused on neutral atom arrays and quantum simulations.
     - **Pasqal**: A French company working on neutral atom quantum processors.

### 7. **Spin Qubits (in diamond or other materials)**
   - **Description**: These qubits use the spin of electrons or nuclei in materials like diamond (NV centers) or silicon. They have long coherence times.
   - **Universities/Institutions**:
     - **University of Chicago**: Leading work on NV center spin qubits in diamond.
     - **Harvard University**: Engaged in research on quantum memories and spin qubits.
     - **University of Stuttgart**: Active in NV centers and spin qubits in diamond.
     - **Element Six**: A company working on synthetic diamond qubits for quantum computing.

These universities and institutions are at the forefront of developing various qubit technologies, each with its own potential benefits and challenges in advancing quantum computing."""